In [1]:
import networkx as nx
import random
import matplotlib.pyplot as plt
from tqdm import tqdm

ImportError: cannot import name 'cumulative_prod' from 'numpy._core' (D:\Study\network-structures-analysis\venv\Lib\site-packages\numpy\_core\__init__.py)

In [ ]:
def calculate_node_metrics(graph: nx.Graph):
    """
    Calculate node metrics for the graph.

    Args:
        graph (nx.Graph): The input graph.

    Returns:
        dict: A dictionary with calculated metrics (degree, centrality, closeness, betweenness).
    """
    print("Calculating node metrics...")
    
    # Degree Centrality
    print("Calculating degree centrality...")
    degree_centrality = dict(graph.degree())
    
    # Approximate Betweenness Centrality
    print("Calculating approximate betweenness centrality...")
    node_betweenness = nx.betweenness_centrality(graph, k=500, seed=42)
    
    # Closeness Centrality
    print("Calculating closeness centrality...")
    node_closeness = nx.closeness_centrality(graph)
    
    # Degree Centrality (normalized)
    print("Calculating normalized degree centrality...")
    node_degree_centrality = nx.degree_centrality(graph)
    
    print("Node metrics calculation completed!")
    return {
        "degree": degree_centrality,
        "betweenness": node_betweenness,
        "closeness": node_closeness,
        "degree_centrality": node_degree_centrality,
    }

def attack_graph_optimized(graph: nx.Graph, attack_strategy, metrics: dict, steps: int = 30):
    """
    Perform an attack on the graph using a given strategy.

    Args:
        graph (nx.Graph): The graph to attack.
        attack_strategy (function): The attack strategy function.
        metrics (dict): Precomputed node metrics.
        steps (int): Number of attack steps.

    Returns:
        list: Sizes of the largest connected component after each attack step.
    """
    print(f"Running attack strategy: {attack_strategy.__name__}")
    giant_component_sizes = []
    initial_size = graph.number_of_nodes()

    for _ in tqdm(range(steps), desc=f"Attacking with {attack_strategy.__name__}"):
        if len(graph) == 0:
            break

        node_to_remove = attack_strategy(graph, metrics)
        graph.remove_node(node_to_remove)

        if len(graph) > 0:
            giant_component = max(nx.connected_components(graph), key=len)
            giant_component_sizes.append(len(giant_component) / initial_size)
        else:
            giant_component_sizes.append(0)

    return giant_component_sizes

def random_failure(graph: nx.Graph, metrics: dict) -> str:
    """Randomly selects a node to remove."""
    return random.choice(list(graph.nodes()))

def highest_degree(graph: nx.Graph, metrics: dict) -> str:
    """Removes the node with the highest degree."""
    return max(metrics["degree"], key=metrics["degree"].get)

def highest_betweenness(graph: nx.Graph, metrics: dict) -> str:
    """Removes the node with the highest betweenness centrality."""
    return max(metrics["betweenness"], key=metrics["betweenness"].get)

def highest_closeness(graph: nx.Graph, metrics: dict) -> str:
    """Removes the node with the highest closeness centrality."""
    return max(metrics["closeness"], key=metrics["closeness"].get)

def plot_attack_results(results: dict, steps: int):
    """
    Plot the results of different attack strategies.

    Args:
        results (dict): A dictionary containing attack results.
        steps (int): Number of attack steps.
    """
    plt.figure(figsize=(12, 8))
    for label, sizes in results.items():
        plt.plot(range(steps), sizes, label=label)

    plt.xlabel("Number of removed nodes")
    plt.ylabel("Relative size of giant component")
    plt.title("Comparison of Attack Strategies")
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# Load the graph
G = nx.read_edgelist("CA-AstroPh.txt", create_using=nx.Graph())

# Precompute metrics
metrics = calculate_node_metrics(G)

# Define attack strategies
strategies = {
    "Random Failure": random_failure,
    "Highest Degree": highest_degree,
    "Highest Betweenness": highest_betweenness,
    "Highest Closeness": highest_closeness,
}

# Run attacks and collect results
results = {}
steps = 30

for label, strategy in strategies.items():
    graph_copy = G.copy()
    results[label] = attack_graph_optimized(graph_copy, strategy, metrics, steps)

# Plot results
plot_attack_results(results, steps)